Install the packages if they are not installed yet

In [1]:
import Pkg
Pkg.add(["COBREXA", "GLPK"])

using COBREXA

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


Let's download and open a big model

In [2]:
import Downloads
Downloads.download("http://bigg.ucsd.edu/static/models/iJO1366.json", "ecoli.json")
m = load_model(StandardModel, "ecoli.json");

...we've opened it as StandardModel right away to allow easy manual
modifications.

see the objective function

In [3]:
objective(m) # gives a sparse vector that maximizes a single reaction
reactions(m)[19] # the number may differ

"BIOMASS_Ec_iJO1366_core_53p95M"

See the exchanges.
Unfortunately, instead of proper annotation the "standard" for identifying
exchange reactions now depends on prefixing the reaction ID with EX_, so at
least we can filter them out by checking the prefixes.

In [4]:
exchanges = filter(startswith("EX_"), reactions(m))

exchanges .=> reaction_name.(Ref(m), exchanges)

324-element Vector{Pair{String, String}}:
        "EX_cm_e" => "Chloramphenicol exchange"
       "EX_cmp_e" => "CMP exchange"
       "EX_co2_e" => "CO2 exchange"
   "EX_cobalt2_e" => "Co2+ exchange"
    "EX_colipa_e" => "Core oligosaccharide lipid A exchange"
    "EX_glc__D_e" => "D-Glucose exchange"
      "EX_glcn_e" => "D-Gluconate exchange"
      "EX_glcr_e" => "D-Glucarate exchange"
   "EX_colipap_e" => "Core oligosaccharide lipid A diphosphate exchange"
     "EX_glcur_e" => "D-Glucuronate exchange"
                  ⋮
  "EX_galct__D_e" => "D-Galactarate exchange"
 "EX_galctn__D_e" => "D-Galactonate exchange"
 "EX_galctn__L_e" => "L-Galactonate exchange"
      "EX_galt_e" => "Galactitol exchange"
     "EX_galur_e" => "D-Galacturonate exchange"
       "EX_gam_e" => "D-Glucosamine exchange"
     "EX_gam6p_e" => "D-Glucosamine 6-phosphate exchange"
     "EX_gbbtn_e" => "Gamma-butyrobetaine exchange"
       "EX_gdp_e" => "GDP exchange"

Let's run the FBA. First we need everyone's favorite linear solver
Ref: https://lcsb-biocore.github.io/COBREXA.jl/stable/examples/05a_fba/

In [5]:
using GLPK

For simplicity, let's ask for a dictionary right away.

In [6]:
sol = flux_balance_analysis_dict(m, GLPK.Optimizer)

Dict{String, Float64} with 2583 entries:
  "Zn2tex"        => 0.000334989
  "GUI1"          => 0.0
  "DXYLK"         => 0.0
  "CBL1tonex"     => 0.0
  "FE3DCITtonex"  => 0.0
  "FACOAL180t2pp" => 0.0
  "METSOXR1"      => 0.0
  "LIPOtex"       => 0.0
  "NTD11"         => 0.0
  "GLUNpp"        => 0.0
  "ORNDC"         => 0.0
  "ALAGLUE"       => 0.0
  "UAGCVT"        => 0.0272981
  "I2FE2ST"       => 2.75064e-5
  "BMOCOS"        => 0.000119849
  "GLCURt2rpp"    => 0.0
  "EX_g3pg_e"     => 0.0
  "EX_acolipa_e"  => 0.0
  "PGSA141"       => 0.0
  ⋮               => ⋮

we can have a look at our objective reaction

In [7]:
sol["BIOMASS_Ec_iJO1366_core_53p95M"]  # (it is possible to tab through the dictionary keys)

0.9823718127269752

we can have a look at the main things that happen

In [8]:
flux_summary(sol)

Biomass
  BIOMASS_Ec_iJO1366_core_53p95M: 0.9824
  BIOMASS_Ec_iJO1366_WT_53p95M:   0.0
Import
  EX_o2_e:                        -17.5789
  EX_nh4_e:                       -10.6104
  EX_glc__D_e:                    -10.0
  EX_pi_e:                        -0.9476
  EX_so4_e:                       -0.2478
  EX_k_e:                         -0.1918
  EX_fe2_e:                       -0.0158
  EX_mg2_e:                       -0.0085
  EX_ca2_e:                       -0.0051
  EX_cl_e:                        -0.0051
  EX_cu2_e:                       -0.0007
  EX_mn2_e:                       -0.0007
  EX_zn2_e:                       -0.0003
  EX_ni2_e:                       -0.0003
  EX_mobd_e:                      -0.0001
  EX_cobalt2_e:                   -0.0
Export
  EX_meoh_e:                      0.0
  EX_h_e:                         9.0263
  EX_co2_e:                       19.6752
  EX_h2o_e:                       45.6194


(this guesses the exchange/biomass status based on reaction IDs)

...or write the result to a file for future use. First, let's create a data frame:

In [9]:
Pkg.add(["DataFrames", "CSV"])
using DataFrames, CSV
df = DataFrame(reaction = collect(keys(sol)), flux = collect(values(sol)))

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


Row,reaction,flux
,String,Float64
1,Zn2tex,0.000334989
2,GUI1,0.0
3,DXYLK,0.0
4,CBL1tonex,0.0
5,FE3DCITtonex,0.0
6,FACOAL180t2pp,0.0
7,METSOXR1,0.0
8,LIPOtex,0.0
9,NTD11,0.0


...and write the CSV:

In [10]:
CSV.write("mysolution.csv", df)

"mysolution.csv"

Let's choke the model a bit, reducing the availbale oxygen and sugar

In [11]:
m.reactions["EX_o2_e"].lb = -20
m.reactions["EX_glc__D_e"].lb = -2
sol = flux_balance_analysis_dict(m, GLPK.Optimizer)
sol["BIOMASS_Ec_iJO1366_core_53p95M"] # less growth

0.18602227486090275

allow eating acetate instead

In [12]:
m.reactions["EX_ac_e"].lb = -100
sol = flux_balance_analysis_dict(m, GLPK.Optimizer)
sol["BIOMASS_Ec_iJO1366_core_53p95M"] # a lot of growth again

0.6895707851538995

At this point, the original model data has been overwritten and there's no
telling which bounds are still from the original model or which have been
modified. For many reasons it is better to do this stuff without breaking the
model internals manually, and COBREXA has a system of "analysis
modifications" for that purpose:

In [13]:
m = load_model(StandardModel, "ecoli.json");
sol = flux_balance_analysis_dict(
    m,
    GLPK.Optimizer,
    modifications = [
        change_constraint("EX_o2_e", lb = -20.0),
        change_constraint("EX_glc__D_e", lb = -2.0),
    ],
);
sol["BIOMASS_Ec_iJO1366_core_53p95M"]
#...this scales much better if you need to try more stuff. Other modifications

0.18602227486090275

include e.g. `change_objective`, `silence` for shutting down the output from
overly verbose solvers (such as OSQP), and `change_optimizer_attribute` for
tuning the optimizer behavior.

There is a nice app at https://escher.github.io/ that allows us to visualize
and browse the solutions to metabolic models. You can load the visualization
of this model as Map: "Core metabolism (e_coli_core)" and Tool: "Viewer".

Let's produce a JSON file with our solution that we can upload:

In [14]:
Pkg.add("JSON")
using JSON

write(
    "mysolution.json",
    JSON.json(
        sol, # the solution
        2, # make the JSON human-readable by using 2-space indentation, instead of optimizing for size
    )
)

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


55523

You can now upload the file `mysolution.json` to the Escher viewer via
`Data → Load reaction data`
to see the fluxes visualized.

More configurable Escher plotting directly from Julia to files (PDF, PNG) is
available via https://github.com/stelmo/Escher.jl

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*